In [93]:
# import PIL
# from tensorflow.keras.preprocessing.image import load_img
# import tensorflow as tf
# import os
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os

In [94]:
# model = Sequential()

# model.add(Conv2D(32, (3, 3), activation='relu', input_shape = (32, 32, 3)))

# model.add(MaxPooling2D(2, 2))

# model.add(Conv2D(64, (3, 3), activation= 'relu'))

# model.add(MaxPooling2D(2, 2))

# model.add(Flatten())

# model.add(Dense(64, activation= 'relu'))

# model.add(Dense(7, activation= 'softmax'))

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [95]:
# model.summary()

data_dir = r"C:\Users\Admin\OneDrive\Desktop\commai-django\commai\intelligence\DATASET"

train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')

train_dataset = ImageFolder(root=train_dir, transform=transform)
test_dataset = ImageFolder(root=test_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("Class mapping:", train_dataset.class_to_idx)
print("Train dataset size:", len(train_dataset))

Class mapping: {'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6}
Train dataset size: 12271


In [96]:
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

import torch.nn as nn
import torch.nn.functional as F

class EmotionCNN(nn.Module):
    def __init__(self):
        super(EmotionCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)

        self.fc1 = nn.Linear(4608, 256)  # Changed from 2048 to 4608
        self.fc2 = nn.Linear(256, 7)  # Assuming 7 output classes

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)

        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)

        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)

        # print(f"Shape before flattening: {x.shape}")  # Debugging

        x = x.view(x.size(0), -1)  # Flatten dynamically
        # print(f"Shape after flattening: {x.shape}")  # Debugging

        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [97]:
# train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

model = EmotionCNN()

In [98]:
# Manually set the correct path
# dataset_path = r"C:\Users\Admin\OneDrive\Desktop\commai-django\commai\intelligence\DATASET\train"

# # Check if the folder exists
# if not os.path.exists(dataset_path):
#     raise FileNotFoundError(f"Dataset folder not found at: {dataset_path}")

# # Load the dataset
# train_data = train_datagen.flow_from_directory(dataset_path, target_size=(32, 32), color_mode="grayscale", class_mode='sparse')

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [99]:
# model.fit(train_data, epochs=10)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch [{epoch+1} / {num_epochs}], Loss: {total_loss:.4f}")

Epoch [1 / 10], Loss: 509.3966
Epoch [2 / 10], Loss: 366.1695
Epoch [3 / 10], Loss: 305.6326
Epoch [4 / 10], Loss: 261.6236
Epoch [5 / 10], Loss: 221.5525
Epoch [6 / 10], Loss: 187.1897
Epoch [7 / 10], Loss: 148.3020
Epoch [8 / 10], Loss: 112.7736
Epoch [9 / 10], Loss: 81.8255
Epoch [10 / 10], Loss: 58.3827


In [100]:
torch.save(model.state_dict(), 'emotion_cnn.pth')

In [102]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F

# Define the same model architecture
class EmotionCNN(nn.Module):
    def __init__(self):
        super(EmotionCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 6 * 6, 256)
        self.fc2 = nn.Linear(256, 7)  # 7 classes

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 128 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Load model
model = EmotionCNN()
model.load_state_dict(torch.load(r"C:\Users\Admin\OneDrive\Desktop\commai-django\commai\intelligence\emotion_cnn.pth"))
model.eval()  # Set model to evaluation mode

EmotionCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4608, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=7, bias=True)
)

In [103]:
# Define the same preprocessing as during training
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.Resize((48, 48)),  # Resize to match training input
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize
])

In [104]:
def preprocess_image(image_path):
    image = Image.open(image_path).convert("L")  # Convert to grayscale
    image = transform(image)  # Apply transformations
    image = image.unsqueeze(0)  # Add batch dimension
    return image

In [105]:
def predict_emotion(image_path, model):
    image = preprocess_image(image_path)
    with torch.no_grad():
        output = model(image)
        predicted_class = torch.argmax(output, dim=1).item()
    return predicted_class

In [110]:
image_path = r"C:\Users\Admin\OneDrive\Desktop\commai-django\commai\intelligence\DATASET\train\7\train_09753_aligned.jpg"
predicted_class = predict_emotion(image_path, model)

class_mapping = {0: "Surprised", 1: "Fear", 2: "Disgust", 3: "Happy", 4: "Sad", 5: "Angry", 6: "Neutral"}
print(f"Predicted Emotion: {class_mapping[predicted_class]}")

Predicted Emotion: Neutral
